In [1]:
import os
import json
import tqdm
import anthropic
import polars as pl
from pathlib import Path
from rich.console import Console
from datasets import load_dataset, load_from_disk
from dotenv import load_dotenv
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    f1_score, 
    precision_score, 
    recall_score
)
from tqdm.auto import tqdm

load_dotenv()
console = Console()

In [2]:
load_dotenv("../.env")

console = Console(style="bold green")

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=ANTHROPIC_API_KEY
)

In [3]:
dataset = load_from_disk("../data/MDACE/mdace-inpatient-icd10")
console.print(dataset)

DatasetDict({
    train: Dataset({
        features: ['hadm_id', 'comment', 'note_id', 'category', 'description', 'text', 'begin', 'end', 
'covered_text', 'code', 'code_system', 'code_description', 'type', 'split'],
        num_rows: 2411
    })
    dev: Dataset({
        features: ['hadm_id', 'comment', 'note_id', 'category', 'description', 'text', 'begin', 'end', 
'covered_text', 'code', 'code_system', 'code_description', 'type', 'split'],
        num_rows: 764
    })
    test: Dataset({
        features: ['hadm_id', 'comment', 'note_id', 'category', 'description', 'text', 'begin', 'end', 
'covered_text', 'code', 'code_system', 'code_description', 'type', 'split'],
        num_rows: 761
    })
})

In [4]:
dataset['train'][1]['covered_text']

'Biliary obstruction'

## Call Anthropic API from Python

In [25]:
def get_icd10_code_anthropic_api(input_text):
    prompt = f"""
    You are a helpful medical coder assistant. 
    Given the following input phrase, provide the most appropriate ICD-10 code, description, and reasoning for your choice.

    Input: {input_text}
    Provide your response in the following JSON format:
    {{
        "code": "<ICD-10 code>",
        "desription": "<Description of the code>",
        "reasoning": "<Your reasoning for selecting this code>"
    }}
    """
    
    response = client.beta.messages.create(
        model="claude-sonnet-4-5",
        max_tokens=1000,
        messages=[{
            "role": "user",
            "content": prompt
        }],
        betas=["structured-outputs-2025-11-13"],
        output_format={
            "type": "json_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "code": {"type": "string"},
                    "desription": {"type": "string"},
                    "reasoning": {"type": "string"},
                },
                "required": ["code", "desription", "reasoning"],
                "additionalProperties": False
            }
        }
    )
    return response

In [ ]:
response = get_icd10_code_anthropic_api(dataset['train'][1]['covered_text'])

In [7]:
console.print(json.loads(response.content[0].text))

{
    'code': 'K83.1',
    'desription': 'Obstruction of bile duct',
    'reasoning': "K83.1 is the most appropriate code for biliary obstruction as it specifically covers obstruction 
of the bile duct. This code is used when there is a blockage in the biliary system without further specification of
the underlying cause (such as choledocholithiasis, neoplasm, or stricture). If the specific etiology of the 
obstruction were known, a more specific code would be selected, but with only 'biliary obstruction' provided, K83.1
is the correct unspecified obstruction code."
}

In [8]:
predicted_code = json.loads(response.content[0].text)['code']
true_code = dataset['train'][1]['code']
console.print(f"Predicted ICD-10 Code: {predicted_code}")
console.print(f"True ICD-10 Code: {true_code}")
assert predicted_code == true_code, "The predicted code does not match the true code."

Predicted ICD-10 Code: K83.1

True ICD-10 Code: K83.1

## LiteLLM API

In [5]:
import litellm
from litellm import completion

In [12]:
def get_icd10_code(input_text):
    prompt = f"""
    You are a helpful medical coder assistant. 
    Given the following input phrase, provide the most appropriate ICD-10 code, description, and reasoning for your choice.

    Input: {input_text}
    Provide your response in the following JSON format:
    {{
        "code": "<ICD-10 code>",
        "desription": "<Description of the code>",
        "reasoning": "<Your reasoning for selecting this code>"
    }}
    """
    
    response = completion(
        model="claude-sonnet-4-5-20250929",
        messages=[{"role": "user", "content": prompt}],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "icd10_response",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "code": {"type": "string"},
                        "description": {"type": "string"},
                        "reasoning": {"type": "string"},
                    },
                    "required": ["code", "desription", "reasoning"],
                    "additionalProperties": False
                }
            }
        }
    )
    return response

In [13]:
response = get_icd10_code(dataset['train'][1]['covered_text'])
print(response)

ModelResponse(id='chatcmpl-94587ba3-1ed3-4362-9ab2-f8008b43e27b', created=1769657313, model='claude-sonnet-4-5-20250929', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='{"code": "K83.1", "description": "Obstruction of bile duct", "reasoning": "K83.1 is the most appropriate code for biliary obstruction as it specifically covers obstruction of the bile duct without specification of the underlying cause. This code is used when there is blockage of the biliary system but no additional details are provided about etiology (such as stones, stricture, or neoplasm). If more specific information were available about the cause or location of the obstruction, a more specific code could be assigned, but based on the general term \'biliary obstruction\' alone, K83.1 is the correct choice."}', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'citations': None, 'thinking_blocks': None}))], usag

In [22]:
predicted_code= json.loads(response.choices[0].message.content)['code']
true_code = dataset['train'][1]['code']
console.print(f"Predicted ICD-10 Code: {predicted_code}")
console.print(f"True ICD-10 Code: {true_code}")
assert predicted_code == true_code, "The predicted code does not match the true code."

Predicted ICD-10 Code: K83.1

True ICD-10 Code: K83.1

## Predictions over Dataset

In [28]:
def predict_split(split_name: str = "dev", limit: int = 50):
    split = dataset[split_name]
    n = min(limit, len(split))
    
    y_true = []
    y_pred = []
    errors = 0
    
    for i in tqdm(range(n), desc=f"Evaluating {split_name}"):
        row = split[i]
        y_true.append(row["code"])
        
        try:
            response = get_icd10_code(row["covered_text"])
            predicted = json.loads(response.choices[0].message.content).get("code", "__MISSING__")
        except Exception as exc:
            errors += 1
            predicted = "__ERROR__"
            console.print(f"[red]Prediction failed at index {i}: {exc}[/red]")
        
        y_pred.append(predicted)

    return y_true, y_pred, errors, n

In [29]:
results = predict_split("dev", limit=25)

Evaluating dev:   0%|          | 0/25 [00:00<?, ?it/s]

In [30]:
def eval_results(y_true, y_pred, errors, n):
    accuracy = accuracy_score(y_true, y_pred)
    macro_precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
    macro_recall = recall_score(y_true, y_pred, average="macro", zero_division=0) 
    macro_f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
    micro_precision = precision_score(y_true, y_pred, average="micro", zero_division=0)
    micro_recall = recall_score(y_true, y_pred, average="micro", zero_division=0) 
    micro_f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)
    weighted_f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    
    console.print()
    
    report = classification_report(y_true, y_pred, zero_division=0)
    results = {
        "evaluated_examples": len(y_true),
        "accuracy": round(accuracy, 4),
        "micro_f1": round(micro_f1, 4),
        "macro precision": round(macro_precision, 4),
        "micro precision": round(micro_precision, 4),
        "micro recall": round(micro_recall, 4),
        "macro recall": round(macro_recall, 4),
        "macro_f1": round(macro_f1, 4),
        "weighted_f1": round(weighted_f1, 4),
    }
    
    return results, report

In [31]:
scores, report = eval_results(*results)
console.print("[bold green]Evaluation Results:[/bold green]")
console.print(scores)
console.print("[bold green]Classification Report:[/bold green]")
console.print(report)

Evaluation Results:

{
    'evaluated_examples': 25,
    'accuracy': 0.72,
    'micro_f1': 0.72,
    'macro precision': 0.5625,
    'micro precision': 0.72,
    'micro recall': 0.72,
    'macro recall': 0.5625,
    'macro_f1': 0.5625,
    'weighted_f1': 0.72
}

Classification Report:

              precision    recall  f1-score   support

       B18.2       1.00      1.00      1.00         1
     C50.919       0.00      0.00      0.00         1
       D05.1       0.00      0.00      0.00         0
       D63.1       0.00      0.00      0.00         1
       D64.9       0.00      0.00      0.00         0
       D69.6       1.00      1.00      1.00         1
      E78.00       1.00      1.00      1.00         1
       E87.2       1.00      1.00      1.00         1
      F10.10       1.00      1.00      1.00         1
     F17.200       0.00      0.00      0.00         1
       F32.9       0.00      0.00      0.00         0
       F32.A       0.00      0.00      0.00         1
         I10       0.00      0.00      0.00         0
       I13.0       0.00      0.00      0.00         1
      I25.10       1.00      1.00      1.00         1
      I50.33       1.00      1.00      1.00         1
       I95.9       1.00      1.00      1.00         1
       J44.1       1.00      1.00      1.00         1
       J44.9       1.00      1.00      1.00         1
      M62.82       1.00      1.00      1.00         1
       N17.9       1.00      1.00      1.00         1
       N18.3       1.00      1.00      1.00         1
      R09.02       1.00      1.00      1.00         1
       R57.1       0.00      0.00      0.00         1
       R57.9       0.00      0.00      0.00         0
       S14.0       0.00      0.00      0.00         0
    S14.109A       0.00      0.00      0.00         1
         Z66       1.00      1.00      1.00         1
       Z72.0       0.00      0.00      0.00         0
     Z87.891       1.00      1.00      1.00         1
       Z95.1       1.00      1.00      1.00         1
      Z99.81       1.00      1.00      1.00         1

    accuracy                           0.72        25
   macro avg       0.56      0.56      0.56        25
weighted avg       0.72      0.72      0.72        25

In [33]:
import json

# Save scores as JSON
with open("results/test-run-1/scores.json", "w") as f:
    json.dump(scores, f, indent=4)

# Save report as text file
with open("results/test-run-1/report.txt", "w") as f:
    f.write(report)

## Trackio

In [34]:
from huggingface_hub import notebook_login, whoami
notebook_login()
console.print(whoami())

{
    'type': 'user',
    'id': '624218d12ddd3fb0a7cae0b8',
    'name': 'schorndorfer',
    'fullname': 'William Thompson',
    'email': 'will.k.t@gmail.com',
    'emailVerified': True,
    'canPay': False,
    'billingMode': 'prepaid',
    'periodEnd': 1769904000,
    'isPro': True,
    'avatarUrl': '/avatars/acb3ff7eae87034f45d3ace8a310544f.svg',
    'orgs': [
        {
            'type': 'org',
            'id': '681b0cb0dba891d54be0773d',
            'name': 'mcp-course',
            'fullname': 'Hugging Face MCP Course',
            'email': None,
            'canPay': False,
            'billingMode': 'postpaid',
            'periodEnd': None,
            'avatarUrl': 
'https://cdn-avatars.huggingface.co/v1/production/uploads/62d648291fa3e4e7ae3fa6e8/itgTDqMrnvgNfJZJ4YmCt.png',
            'roleInOrg': 'read',
            'isEnterprise': False
        }
    ],
    'auth': {
        'type': 'access_token',
        'accessToken': {'displayName': 'test-token', 'role': 'write', 'createdAt': '2025-12-25T15:54:04.858Z'}
    }
}

In [35]:
scores = json.load(open("results/test-run-1/scores.json"))
console.print(scores)

{
    'evaluated_examples': 25,
    'accuracy': 0.72,
    'micro_f1': 0.72,
    'macro precision': 0.5625,
    'micro precision': 0.72,
    'micro recall': 0.72,
    'macro recall': 0.5625,
    'macro_f1': 0.5625,
    'weighted_f1': 0.72
}

In [41]:
import trackio

trackio.init(project="my-project", space_id="schorndorfer/test-space")

* Created new run: schorndorfer-1769658442


In [42]:
for i in range(10):
    trackio.log({"loss": 1/(i+1)})

In [43]:
df = pl.DataFrame(scores)
df

evaluated_examples,accuracy,micro_f1,macro precision,micro precision,micro recall,macro recall,macro_f1,weighted_f1
i64,f64,f64,f64,f64,f64,f64,f64,f64
25,0.72,0.72,0.5625,0.72,0.72,0.5625,0.5625,0.72


In [44]:
trackio.log(
    {
        "scores": trackio.Table(dataframe=df.to_pandas()),
    }
)

In [45]:
trackio.finish()

* Run finished. Uploading logs to Trackio (please wait...)
